# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_key import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_data_df = pd.read_csv('../pandas_api_challenge/city_data.csv')
 

Cities        565
Country       562
Latitude      565
Longitude     565
Tempature     565
Humidity      565
Cloudiness    565
Wind Speed    565
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
locations = city_data_df[["Latitude", "Longitude"]]

humidity = city_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
city_data_df.head()

,Cities,Country,Latitude,Longitude,Tempature,Humidity,Cloudiness,Wind Speed
0,alice springs,AU,-23.7000,133.8833,8.00,61,0,2.06
1,san cristobal,VE,7.7669,-72.2250,26.00,78,75,2.57
2,albany,US,42.6001,-73.9662,20.88,36,90,1.39
3,asau,RO,46.4333,26.4000,6.11,61,100,1.03
4,mataura,NZ,-46.1927,168.8643,11.67,91,99,0.89


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
temp_df = city_data_df.loc[(city_data_df["Tempature"] > 25)]
                           
wind_df =temp_df.loc[temp_df["Wind Speed"] < 3] 

final_df = wind_df.loc[wind_df["Cloudiness"] < 3.9]

final_df = final_df.reset_index()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
target_search = "Hotel"
radius = 5000

hotels = []
#lat = final_df["Latitude"]
#lng= final_df["Longitude"]


for index, row in final_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    params = {
        "location": f"{lat},{lng}",
        "keyword": target_search,
        "radius": radius,
        "key": g_key}

# Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



# Run request
    hotel_data = requests.get(base_url, params).json()
    
    
    hotels.append(hotel_data["results"][0]["name"])

    
    
    




In [62]:
cities = final_df["Cities"]

hotels_df = pd.DataFrame({"Cities":cities,"Hotel Name":hotels})
hotels_df


final_hotel_df = final_df.merge(hotels_df, on = "Cities")


In [64]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]
locations = final_hotel_df[["Latitude", "Longitude"]]

In [65]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))